In [563]:
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#Imports pickle
import pickle

#Library for indicators
from finta import TA

#Another library for indicators
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import PSARIndicator
from ta.trend import SMAIndicator
from ta.trend import macd
from ta.momentum import rsi
from ta.volume import OnBalanceVolumeIndicator

#Import yfinance
import yfinance as yf

#Import yfinance
import tensorflow as tf


In [566]:
#Downloads info from Yahoo Finance
df = yf.download(tickers="BTC-USD",
                            period="60d",
                            interval="15m",
                            auto_adjust=True)


#Sort values from past to present
df = df.reset_index(drop=True)

df = df.drop(columns=['Volume'])


#Indicators with TA
#Adds simple moving average
df["sma7"] = SMAIndicator(close=df["Open"], window=7, fillna=True).sma_indicator()
df["sma25"] = SMAIndicator(close=df["Open"], window=25, fillna=True).sma_indicator()
df["sma99"] = SMAIndicator(close=df["Open"], window=99, fillna=True).sma_indicator()

#Adds Bollinger Bands
indicator_bb = BollingerBands(close=df["Open"], window=20, window_dev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

#Adds PSAR Indicator
indicator_psar = PSARIndicator(high=df["Close"], low=df["Low"], close=df["Close"], step=0.02, max_step=2, fillna=True)
df['psar'] = indicator_psar.psar()

# Add Moving Average Convergence Divergence (MACD) indicator
df["MACD"] = macd(close=df["Close"], window_slow=26, window_fast=12, fillna=True)
    
# Add Relative Strength Index (RSI) indicator
df["RSI"] = rsi(close=df["Close"], window=14, fillna=True)

#Indicators using finta
#Adds simple moving average
df["sma40"] = TA.SMA(df, period = 40)

# Add Relative Strength Index (RSI) indicator
#df["OBV"] = TA.OBV(df)

#Drops empty spaces
df.fillna(0, inplace=True)

#Drops empty spaces
df.head(50)


df = df.drop(['Low','Close','High'], axis=1)


df

[*********************100%***********************]  1 of 1 completed


/Users/esteban/miniforge3/envs/tf_m1/lib/python3.8/site-packages/ta/trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
/Users/esteban/miniforge3/envs/tf_m1/lib/python3.8/site-packages/ta/trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Series(index=self._psar.index)


,Open,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40
0,19302.492188,19302.492188,19302.492188,19302.492188,0.000000,0.000000,0.000000,19305.281250,0.000000,100.000000,0.000000
1,19311.595703,19307.043945,19307.043945,19307.043945,0.000000,0.000000,0.000000,19267.486328,-3.014980,0.000000,0.000000
2,19278.017578,19297.368490,19297.368490,19297.368490,0.000000,0.000000,0.000000,19305.281250,-6.899742,0.000000,0.000000
3,19249.294922,19285.350098,19285.350098,19285.350098,0.000000,0.000000,0.000000,19304.087422,-8.261186,28.386622,0.000000
4,19267.144531,19281.708984,19281.708984,19281.708984,0.000000,0.000000,0.000000,19301.584316,-16.261434,12.132185,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
5734,19835.541016,19747.051897,19886.166562,20021.862630,19846.944336,20076.715466,19617.173206,19657.582383,-58.151748,46.655810,19903.895996
5735,19810.087891,19751.110212,19878.108906,20019.386758,19839.597852,20064.158188,19615.037515,19661.123743,-57.098779,41.803589,19899.931641
5736,19753.037109,19744.960100,19865.161172,20015.630583,19826.237305,20037.625529,19614.849080,19664.594276,-53.877467,43.918938,19896.710107
5737,19774.130859,19746.429408,19852.872188,20012.344243,19814.726855,20010.568458,19618.885253,19667.995398,-48.089263,47.065021,19893.822803


In [567]:
#Loads model

model = tf.keras.models.load_model('Classifier_NN_hourly_v2.0')

#Makes the prediction

y_pred = model.predict(df.iloc[-1:].to_numpy().reshape(1,-1))

#Makes the prediction

print(y_pred[0])

df.iloc[-1:]

[1.]


,Open,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40
5738,19828.734375,19769.687221,19843.837969,20010.876559,19807.367871,19988.972644,19625.763098,19671.328498,-40.92165,49.466317,19892.059326
